In [35]:
# US by_state daily cases with 3 days preceding and 3 days following(3P3F) average 

import datetime
import time
from google.cloud import bigquery  # This is for running sql to get the data
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import (HoverTool, ColumnDataSource,DatetimeTickFormatter,Range1d)
from bokeh.layouts import gridplot

def dataframe_from_sql(col_names, sql_str):
    dict = {}
    for x in col_names:
        dict[x] = []

    client = bigquery.Client(project='paul-henry-tremblay')
    result = client.query(sql_str)

    for rows in result:
        for x in col_names:
            dict[x].append(rows.get(x))
 
    return pd.DataFrame.from_dict(dict)

# by state daily cases and deaths 
def make_state_graph(state, df, max_y=None):
    
    #df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'])
   
    #df.loc[:,'ToolTipDates'] = df.loc[:,'date'].map(lambda x: x.strftime("%b %d"))
    
    df_state = df[df['state'] == state]
  
    # df_state.loc[:,'date'] = pd.to_datetime(df_state.loc[:,'date'])
   
    # df_state.loc[:,'ToolTipDates'] = df_state.loc[:,'date'].map(lambda x: x.strftime("%b %d")) 
    
    source = ColumnDataSource(df_state)
      
    p = figure(plot_height=400, 
               plot_width=500, 
               title='{state} State Daily cases with 7-day Avg 3P3F'.format(state=state), 
               x_axis_label="Date",
               y_axis_label="Daily new cases",
               toolbar_location="below")
    
    
    if max_y is not None:
        p.y_range = Range1d(0, max_y)
    
    p.vbar(x='date', 
           top='daily_cases', 
           bottom = -10, 
           source = source, 
           line_width=1, 
           width=datetime.timedelta(days=0.5), 
           color='blue', 
           legend_label='daily_cases')
        
    p.line(x='date', 
           y='seven_day_cases_3P3F_Average', 
           source = source, line_width=3, 
           color='firebrick', 
           legend_label='seven_day_cases_3P3F_Average')

    p.xaxis.formatter = DatetimeTickFormatter(days=['%m/%d', '%a%d%y'])
    p.legend.location = 'top_left'
    p.xgrid.grid_line_color = None
    p.add_tools(
        HoverTool(tooltips=
                  [('date', '@ToolTipDates'),
                   ('daily_cases', '@daily_cases'),
                   ('7-day_3P3F Average', '@seven_day_cases_3P3F_Average')],
                  mode='vline'))
       
    p.legend.label_text_font_size = '8pt'
    
    return(p)
    
# make all states graphs
def make_states_graph(df):
   
    # Find list of all states then sort this list.
    states = sorted(list(set(df['state'])))
        
    # Find the largest number of cases, exclude the none number if there are.
    # Inflating 5% of the max_cases number to make the graph more elegant.
    max_cases = max(x for x in df['daily_cases'] if pd.notna(x)) * 1.02
      
    p_list = []
    for i in states:
        p_list.append(make_state_graph(i, df, max_y=max_cases))

    grid = gridplot(p_list, ncols=2)
    
    show(grid)
    

                      
output_notebook()
sql_str = """
  
    with dc as (
    select date, 
           state, 
           cases-lag(cases, 1) over(partition by state order by date) as daily_cases
           from covid19.us_states
          order by state, date)
    select date, 
           state,
           daily_cases,
           round(AVG(daily_cases) OVER (partition by state ORDER BY date ROWS BETWEEN 3 PRECEDING AND 3 following),0) 
             AS seven_day_cases_3P3F_Average,
    from dc
    order by state, date;"""
    
col_names = ['date', 'state', 'daily_cases', 'seven_day_cases_3P3F_Average']

df = dataframe_from_sql(col_names, sql_str)
    
df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'])
   
df.loc[:,'ToolTipDates'] = df.loc[:,'date'].map(lambda x: x.strftime("%b %d"))

make_state_graph('Washington', df)

make_states_graph(df)


Loading BokehJS ...

In [37]:
# US by_state daily deaths with 3 days preceding and 3 days following(3P3F) average 

import datetime
import time
from google.cloud import bigquery  # This is for running sql to get the data
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import (HoverTool, ColumnDataSource,DatetimeTickFormatter,Range1d)
from bokeh.layouts import gridplot

def dataframe_from_sql(col_names, sql_str):
    dict = {}
    for x in col_names:
        dict[x] = []

    client = bigquery.Client(project='paul-henry-tremblay')
    result = client.query(sql_str)

    for rows in result:
        for x in col_names:
            dict[x].append(rows.get(x))
 
    return pd.DataFrame.from_dict(dict)

# by state daily cases and deaths 
def make_state_graph(state, df, max_y=None):
    # Filter state    
    df_state = df[df['state'] == state]
    # convert to ColumnDataSource
    source = ColumnDataSource(df_state)
      
    p = figure(plot_height=400, 
               plot_width=500, 
               title='{state} State Daily deaths with 7-day Avg 3P3F'.format(state=state), 
               x_axis_label="Date",
               y_axis_label="Daily new deaths",
               toolbar_location="below")
    
    
    if max_y is not None:
        p.y_range = Range1d(0, max_y)
    
    p.vbar(x='date', 
           top='daily_deaths', 
           bottom = -10, 
           source = source, 
           line_width=1, 
           width=datetime.timedelta(days=0.5), 
           color='blue', 
           legend_label='daily_deaths')
        
    p.line(x='date', 
           y='seven_day_deaths_3P3F_Average', 
           source = source, line_width=3, 
           color='firebrick', 
           legend_label='seven_day_deaths_3P3F_Average')

    p.xaxis.formatter = DatetimeTickFormatter(days=['%m/%d', '%a%d%y'])
    p.legend.location = 'top_left'
    p.xgrid.grid_line_color = None
    p.add_tools(
        HoverTool(tooltips=
                  [('date', '@ToolTipDates'),
                   ('daily_deaths', '@daily_deaths'),
                   ('7-day_3P3F Average', '@seven_day_deaths_3P3F_Average')],
                  mode='vline'))
       
    p.legend.label_text_font_size = '8pt'
    
    return(p)
    
# make all states graphs
def make_states_graph(df):
   
    # Find list of all states then sort this list.
    states = sorted(list(set(df['state'])))
        
    # Find the largest number of cases, exclude the none number if there are.
    # Inflating 1% of the max_cases number to make the graph more elegant.
    max_cases = max(x for x in df['daily_deaths'] if pd.notna(x)) * 1.01
      
    p_list = []
    for i in states:
        p_list.append(make_state_graph(i, df, max_y=max_cases))

    grid = gridplot(p_list, ncols=2)
    
    show(grid)
    

                      
output_notebook()
sql_str = """
  
    with dc as (
    select date, 
           state, 
           deaths-lag(deaths, 1) over(partition by state order by date) as daily_deaths
           from covid19.us_states
          order by state, date)
    select date, 
           state,
           daily_deaths,
           round(AVG(daily_deaths) OVER (partition by state ORDER BY date ROWS BETWEEN 3 PRECEDING AND 3 following),0) 
             AS seven_day_deaths_3P3F_Average,
    from dc
    order by state, date;"""
    
col_names = ['date', 'state', 'daily_deaths', 'seven_day_deaths_3P3F_Average']

df = dataframe_from_sql(col_names, sql_str)
    
df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'])
   
df.loc[:,'ToolTipDates'] = df.loc[:,'date'].map(lambda x: x.strftime("%b %d"))

make_state_graph('Washington', df)

make_states_graph(df)


Loading BokehJS ...

In [1]:
# US nationalwide daily cases and deaths with 7-day trailing average 

import datetime
import time
from google.cloud import bigquery  # This is for running sql to get the data
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import (HoverTool, ColumnDataSource,DatetimeTickFormatter)


def dataframe_from_sql(col_names, sql_str):
    dict = {}
    for x in col_names:
        dict[x] = []

    client = bigquery.Client(project='paul-henry-tremblay')
    result = client.query(sql_str)

    for rows in result:
        for x in col_names:
            dict[x].append(rows.get(x))
 
    return pd.DataFrame.from_dict(dict)

# National wide daily cases and deaths 
def make_US_graph(df):
     
    df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'])
   
    df.loc[:,'ToolTipDates'] = df.loc[:,'date'].map(lambda x: x.strftime("%b %d")) 
    
    source = ColumnDataSource(df)
      
    p = figure(plot_height=600, 
               plot_width=1000, 
               title='Daily cases and deaths - United States', 
               x_axis_label="Date",
               y_axis_label="Daily new cases and deaths",
               toolbar_location="below")

    p.vbar(x='date', 
           top='daily_cases', 
           bottom = -10, 
           source = source, 
           line_width=1, 
           width=datetime.timedelta(days=.5), 
           color='blue', 
           legend_label='daily_cases')
    
    p.vbar(x='date', 
           top='daily_deaths', 
           bottom=-10, 
           source = source, 
           line_width=1, 
           width=datetime.timedelta(days=.5),
           color='red', 
           legend_label='daily_deaths')
    
    p.line(x='date', 
           y='seven_day_cases_trailing_Average', 
           source = source, line_width=3, 
           color='firebrick', 
           legend_label='7-day cases trailing Average')
    
    p.line(x='date', 
           y='seven_day_deaths_trailing_Average', 
           source = source, 
           line_width=3, 
           color='green', 
           legend_label='7-day deaths trailing Average')
    
            
    p.xaxis.formatter = DatetimeTickFormatter(days=['%m/%d', '%a%d%y'])
    p.legend.location = 'top_left'
    p.xgrid.grid_line_color = None
    p.add_tools(
        HoverTool(tooltips=
                  [('date', '@ToolTipDates'),
                   ('daily_cases', '@daily_cases'),
                   ('seven_day_cases_trailing_Average', '@seven_day_cases_trailing_Average'),
                   ('daily_deaths', '@daily_deaths'),
                   ('seven_day_deaths_trailing_Average','@seven_day_deaths_trailing_Average')],
                  mode='vline'))
       
    p.legend.label_text_font_size = '8pt'
    show(p)
    
                      
output_notebook()
sql_str = """
  
    with nc as 
    (select date, sum(cases) as cases,
    sum(deaths) as deaths
    from covid19.us_states
    group by date
    order by date),
    di as (
    select date,
           cases-lag(cases, 1) over(order by date) as daily_cases,
           deaths-lag(deaths,1) over(order by date) as daily_deaths
    from nc
    order by date)
    select date,
           daily_cases,
           round(AVG(daily_cases) OVER (ORDER BY date ROWS BETWEEN 3 PRECEDING AND 3 following),0) AS seven_day_cases_trailing_Average,
           daily_deaths,
           round(AVG(daily_deaths) OVER (ORDER BY date ROWS BETWEEN 3 PRECEDING AND 3 following),0) AS seven_day_deaths_trailing_Average
    from di
    order by date;"""
    
col_names = ['date', 'daily_cases', 'seven_day_cases_trailing_Average', 'daily_deaths', 'seven_day_deaths_trailing_Average']

df = dataframe_from_sql(col_names, sql_str)

make_US_graph(df)


Loading BokehJS ...